# Training and Evaluation

In [ ]:
import cv2 as cv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms


from torch.utils.data import DataLoader
from models.baseline_average import BaselineAverageModel
from models.convolved_average import ConvolvedAverageModel
from models.empty_checkerboard import EmptyCheckerboardModel
from reconstruct import nparray_to_mp4, mp4_to_nparray
from score import calculate_mse, evaluate_and_save_video

%load_ext autoreload
%autoreload 2

In [2]:
IN_HEIGHT = 2160
IN_WIDTH = 1920
OUT_HEIGHT = 2160
OUT_WIDTH = 3840

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.cuda.is_available()

True

In [39]:
input_path = "data/Seconds_That_Count-checkerboard.mov"
output_path_cam = "data/Seconds_That_Count-cvavg.mov"
output_path_bavg = "data/Seconds_That_Count-bavg.mov"
native_path = "data\Seconds_That_Count-native.mov"
empty_checkerboard_path = "data\LifeSeconds_That_Count-checkerboard-empty.mov"

# input_path = "data\Lifting_Off-checkerboard.mov"
# output_path_cam = "data/Lifting_Off-cvavg.mov"
# output_path_bavg = "data/Lifting_Off-bavg.mov"
# native_path = "data\Lifting_Off-native.mov"

# input_path = "data\Moment+of+Intensity-checkerboard.mov"
# output_path_cam = "data/Moment+of+Intensity-cvavg.mov"
# output_path_bavg = "data/Moment+of+Intensity-bavg.mov"
# native_path = "data\Moment+of+Intensity-native.mov"

# input_path = "data\LifeUntouched_P3_4K_PQ_XQ-checkerboard.mov"
# output_path_cam = "data/LifeUntouched_P3_4K_PQ_XQ-cvavg.mov"
# output_path_bavg = "data/LifeUntouched_P3_4K_PQ_XQ-bavg.mov"
# empty_checkerboard_path = "data\LifeUntouched_P3_4K_PQ_XQ-checkerboard-empty.mov"
# output_path_s3dcnn = "data/LifeUntouched_P3_4K_PQ_XQ-s3dcnn.mov"
# native_path = "data\LifeUntouched_P3_4K_PQ_XQ-native.mov"

In [61]:
# Instantiate model
cavg_model = ConvolvedAverageModel(in_shape=(IN_HEIGHT, IN_WIDTH), out_shape=(OUT_HEIGHT, OUT_WIDTH))

In [62]:
cavg_model(input_path, output_path_cam)

Upscaled video saved to data/LifeUntouched_P3_4K_PQ_XQ-cvavg.mov


In [ ]:
bavg_model = BaselineAverageModel(in_shape=(IN_HEIGHT, IN_WIDTH), out_shape=(OUT_HEIGHT, OUT_WIDTH))

In [ ]:
bavg_model(input_path, output_path_bavg)

Finished processing video.


In [ ]:
# evaluate
calculate_mse(native_path, output_path_bavg)


np.float32(0.0)

In [66]:
calculate_mse(native_path, output_path_cam)


np.float32(528975800.0)

In [40]:
empty_checkerboard = EmptyCheckerboardModel(in_shape=(IN_HEIGHT, IN_WIDTH), out_shape=(OUT_HEIGHT, OUT_WIDTH))

In [41]:
empty_checkerboard(input_path, empty_checkerboard_path)

Finished processing video.


In [ ]:
from models.video_dataset import VideoDataset
from models.cnn import CNNInterpolator

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNInterpolator().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

video_dataset = VideoDataset(empty_checkerboard_path, native_path, max_frames=100)
video_dataloader = DataLoader(video_dataset, batch_size=1, shuffle=True)

print("Begin Training")
for epoch in range(10):
    total_loss = 0
    for input_frames, target_frames in video_dataloader:
        input_frames, target_frames = input_frames.to(device), target_frames.to(device)

        optimizer.zero_grad()
        output = model(input_frames)
        loss = criterion(output, target_frames)
        loss.backward()
        optimizer.step()

        del input_frames, target_frames, output
        torch.cuda.empty_cache()

        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(video_dataloader)}")

torch.save(model.state_dict(), "cnn_interpolator.pth")
video_dataset.close()


Begin Training
Epoch 1, Loss: 1557.2991703073308
Epoch 2, Loss: 54.000503236986695
Epoch 3, Loss: 356.50955698251727
Epoch 4, Loss: 199.53098429899663
Epoch 5, Loss: 56.464440801665184
Epoch 6, Loss: 61.39682021178305
Epoch 7, Loss: 32.63146009549499
Epoch 8, Loss: 24.75456662610173
Epoch 9, Loss: 24.46308688789606
Epoch 10, Loss: 205.9574350516498


In [50]:
from models.cnn import CNNReconstructor

reconstructor = CNNReconstructor("cnn_interpolator.pth")
reconstructor.process_video(empty_checkerboard_path, "output_reconstructed.mov")

Reconstructed video saved to output_reconstructed.mov
